# PydanticAI fundamentals

In [2]:
from pydantic_ai import Agent

agent = Agent(model="google-gla:gemini-2.5-flash")

result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result

AgentRunResult(output='**Name:** Erik Larsson\n**Role:** Senior Software Developer\n**Company:** Telia Company\n**Location:** Stockholm, Sweden\n**Focus:** Backend development (Java, Spring Boot) for digital services, cloud infrastructure (AWS), and CI/CD pipelines.')

In [4]:
print(result.output)

**Name:** Erik Larsson
**Role:** Senior Software Developer
**Company:** Telia Company
**Location:** Stockholm, Sweden
**Focus:** Backend development (Java, Spring Boot) for digital services, cloud infrastructure (AWS), and CI/CD pipelines.


## Use pydantic to structure output

In [6]:
from pydantic import BaseModel, Field

class Employee(BaseModel):
    name: str = Field (description= "first name and last name of the person")
    age: int = Field(gt= 17, lt= 67, description = "age of a person")
    salary: int = Field(gt= 30000, lt = 70000, description= "salary of the employee, the more senior the role, the higher the salary")
    role: str = Field(description= "always include seniority level in the role (junior, mid-level, senior, expert)")
    description: str = Field(description = " short description (max 2 sentences) of the role, include keywords")

result = await agent.run("Give me an IT employee working as a data engineer in Sweden", output_type = Employee)

result    


AgentRunResult(output=Employee(name='Erik Svensson', age=32, salary=60000, role='Mid-level Data Engineer', description='Responsible for developing, constructing, testing, and maintaining data architectures such as databases and large-scale processing systems. Works with data pipeline development and optimization.'))

In [8]:
employee = result.output

employee.description, employee.name

('Responsible for developing, constructing, testing, and maintaining data architectures such as databases and large-scale processing systems. Works with data pipeline development and optimization.',
 'Erik Svensson')

In [9]:
isinstance(employee, BaseModel)

True

In [10]:
employee.model_dump()

{'name': 'Erik Svensson',
 'age': 32,
 'salary': 60000,
 'role': 'Mid-level Data Engineer',
 'description': 'Responsible for developing, constructing, testing, and maintaining data architectures such as databases and large-scale processing systems. Works with data pipeline development and optimization.'}

In [12]:
print(employee.model_dump_json(indent=3))

{
   "name": "Erik Svensson",
   "age": 32,
   "salary": 60000,
   "role": "Mid-level Data Engineer",
   "description": "Responsible for developing, constructing, testing, and maintaining data architectures such as databases and large-scale processing systems. Works with data pipeline development and optimization."
}


### Simulate 10 employees

In [15]:
result = await agent.run(
    "give me 10 employees in AI, data science, data engineering and data analytics field",
    output_type=list[Employee])


result

AgentRunResult(output=[Employee(name='Alice Smith', age=30, salary=60000, role='Mid-level AI Engineer', description='Specializes in developing and deploying machine learning models for natural language processing. Experienced with TensorFlow and PyTorch.'), Employee(name='Bob Johnson', age=35, salary=65000, role='Senior Data Scientist', description='Leads data analysis projects, builds predictive models, and derives actionable insights. Proficient in Python, R, and SQL.'), Employee(name='Charlie Brown', age=28, salary=55000, role='Junior Data Engineer', description='Designs and implements robust ETL pipelines and manages data infrastructure. Skilled in Apache Spark, Hadoop, and AWS.'), Employee(name='Diana Miller', age=32, salary=62000, role='Mid-level Data Analyst', description='Analyzes complex datasets to identify trends and create data visualizations. Expertise in Tableau, Power BI, and advanced Excel.'), Employee(name='Eve Davis', age=40, salary=68000, role='Expert AI Researcher',

In [18]:
for employee in result.output:
    print(f"{employee.name} is {employee.age} years old, earns {employee.salary} SEK and has role {employee.role}")

Alice Smith is 30 years old, earns 60000 SEK and has role Mid-level AI Engineer
Bob Johnson is 35 years old, earns 65000 SEK and has role Senior Data Scientist
Charlie Brown is 28 years old, earns 55000 SEK and has role Junior Data Engineer
Diana Miller is 32 years old, earns 62000 SEK and has role Mid-level Data Analyst
Eve Davis is 40 years old, earns 68000 SEK and has role Expert AI Researcher
Frank White is 38 years old, earns 67000 SEK and has role Senior Data Engineering Lead
Grace Lee is 27 years old, earns 58000 SEK and has role Junior Data Scientist
Henry Wilson is 45 years old, earns 69000 SEK and has role Expert Data Analytics Manager
Ivy Moore is 33 years old, earns 63000 SEK and has role Mid-level AI Ethicist
Jack Taylor is 31 years old, earns 61000 SEK and has role Mid-level Machine Learning Engineer


## More complex models

In [21]:
from typing import Literal
class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing"] = Field(description= "Name of the department")
    description: str = Field(description= "Short description of the department")
    employees: list[Employee]

class Company(BaseModel):
    name: str = Field(description="name of the company")
    departments: list[Department]
    description: str = Field(description="describe company in two sentences")

result = await agent.run(
    ("give me 2-3 employees in each department", "Company should be called Cool company and sell ice creams"),
    output_type=Company)


In [25]:
company = result.output
company.__dict__.keys()

dict_keys(['name', 'departments', 'description'])

In [29]:
for department in company.departments:
    print(department)

name='IT' description='The IT department is responsible for managing and maintaining all technology infrastructure and systems, ensuring smooth operations and data security.' employees=[Employee(name='Alice Smith', age=45, salary=68000, role='Senior IT Manager', description='Oversees all IT operations, manages the IT team, and develops IT strategies. Focuses on system reliability and cybersecurity.'), Employee(name='Bob Johnson', age=32, salary=65000, role='Mid-level System Administrator', description='Manages server infrastructure, network devices, and resolves technical issues. Ensures optimal performance of IT systems.'), Employee(name='Charlie Brown', age=24, salary=45000, role='Junior IT Support Specialist', description='Provides first-level technical support to employees, troubleshoots hardware and software problems. Assists with IT onboarding.')]
name='HR' description='The HR department handles all aspects of human resources, including recruitment, employee relations, payroll, a

In [31]:
departments = [department for department in company.departments]
departments[0].employees

[Employee(name='Alice Smith', age=45, salary=68000, role='Senior IT Manager', description='Oversees all IT operations, manages the IT team, and develops IT strategies. Focuses on system reliability and cybersecurity.'),
 Employee(name='Bob Johnson', age=32, salary=65000, role='Mid-level System Administrator', description='Manages server infrastructure, network devices, and resolves technical issues. Ensures optimal performance of IT systems.'),
 Employee(name='Charlie Brown', age=24, salary=45000, role='Junior IT Support Specialist', description='Provides first-level technical support to employees, troubleshoots hardware and software problems. Assists with IT onboarding.')]

In [32]:
company.model_dump()

{'name': 'Cool company',
 'departments': [{'name': 'IT',
   'description': 'The IT department is responsible for managing and maintaining all technology infrastructure and systems, ensuring smooth operations and data security.',
   'employees': [{'name': 'Alice Smith',
     'age': 45,
     'salary': 68000,
     'role': 'Senior IT Manager',
     'description': 'Oversees all IT operations, manages the IT team, and develops IT strategies. Focuses on system reliability and cybersecurity.'},
    {'name': 'Bob Johnson',
     'age': 32,
     'salary': 65000,
     'role': 'Mid-level System Administrator',
     'description': 'Manages server infrastructure, network devices, and resolves technical issues. Ensures optimal performance of IT systems.'},
    {'name': 'Charlie Brown',
     'age': 24,
     'salary': 45000,
     'role': 'Junior IT Support Specialist',
     'description': 'Provides first-level technical support to employees, troubleshoots hardware and software problems. Assists with IT 

In [ ]:
co